# Manipulación y prepración de datos
* Importar los conjuntos de datos con los que se trabajará.
* Realizar tareas de limpieza de datos, eliminando valores nulos o inconsistentes.
* Realizar transformaciones necesarias para preparar los datos para el análisis y modelado.

# Exploración y preprocesamiento de datos
* Realizar análisis exploratorio de datos (EDA) utilizando visualizaciones y estadísticas descriptivas.
* Aplicar técnicas de preprocesamiento como normalización, codificación de variables categóricas y reducción de dimensionalidad.


# Versionado de datos

* Utilizar herramientas de versionado de datos para mantener un registro de cambios y versiones de los conjuntos de datos utilizados.
* Documentar todas las modificaciones realizadas en los datos.

# Construcción, ajuste y evaluación de modelos de machine learning
* Seleccionar algoritmos de Machine Learning adecuados para resolver el problema planteado.
* Entrenar los modelos utilizando los datos preprocesados.
* Realizar ajustes de hiperparámetros para optimizar el rendimiento del modelo.
* Evaluar los modelos utilizando métricas de rendimiento específicas.


# Aplicación de mejores prácticas en el pipeline de modelado
* Implementar pipelines de Machine Learning en donde se automaticen las tareas de preprocesamiento, entrenamiento y evaluación.
* Documentar cada paso del pipeline para asegurar claridad y reproducibilidad.


# Estructuración y refactorización del código
* Organizar el código en módulos y funciones bien definidas.
* Aplicar principios de programación orientada a objetos para mejorar la estructura del código.
* Refactorizar el código existente para mejorar su eficiencia y mantenibilidad.

# Presentación ejectuvia de pase pdf